## HW4_3 template
 BE SURE TO CHANGE VALUES OTHERWISE THIS SCRIPT WILL NOT RUN; this problem can handle a 4 dimensional SIR model

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm, multivariate_normal, invgamma, gaussian_kde
from scipy.integrate import solve_ivp



In [ ]:
# Define initial conditions and problem parameters
param_star = np.array([X, X, X, X])  # Replace with actual values
noise_var = XXX  # Replace with noise variance
s_theta = XXX * np.ones(len(param_star))  # Proposal covariance magnitude
UB_uni = np.array([XXX, XXX, XXX, XXX])  # Upper bounds
LB_uni = np.array([XXX, XXX, XXX, XXX])  # Lower bounds
IC = [XXX,XXX,XXX]  # Replace with initial conditions


# Time and space discretization
tend = 10
tspace = np.linspace(0, tend, 30)
num_param = len(param_star)
n_xpts = len(tspace)


In [ ]:
# Define the QoI (quantity of interest) function
def call_SIR_model(q, IC, tspace):
    """
    Solve the SIR model and return the I(t) state.
    :param q: Parameters [gamma, k, r, mu].
    :param IC: Initial conditions [S0, I0, R0].
    :param tspace: Time points.
    :return: I(t) state as a 1D array.
    """
    N = sum(IC)  # Total population
    result = solve_ivp(SIR_model, [tspace[0], tspace[-1]], IC, t_eval=tspace, args=(q, N))
    return result.y[1]  # Return I(t) state (second entry in y)

def SIR_model(t, y, params, N):
    """
    Define the RHS of the SIR model ODE system.
    :param t: Current time.
    :param y: Current state [S, I, R].
    :param params: Parameters [gamma, k, r, mu].
    :param N: Total population size.
    :return: dy/dt as a list [dS/dt, dI/dt, dR/dt].
    """
    # Unpack parameters
    gamma, k, r, mu = params

    # Redefine state variables
    S, I, R = y

    # RHS equations
    dS = mu * N - mu * S - gamma * k * I * S
    dI = gamma * k * I * S - (r + mu) * I
    dR = r * I - mu * R

    return [dS, dI, dR]


In [ ]:

def Metropolis_errupdate(f, data, prior_F, theta0, M, covar, UB, LB):
    """
    Implements the Metropolis algorithm with error variance updates using inverse gamma.

    Parameters:
    f: function - Model function.
    data: array - Observed data.
    prior_F: function - Prior probability distribution.
    theta0: array - Initial parameter values.
    M: int - Number of samples in the Markov chain.
    covar: array - Covariance matrix for proposal distribution.
    UB: array - Upper bounds for parameters.
    LB: array - Lower bounds for parameters.

    Returns:
    chain: array - Markov chain of sampled parameters.
    s2chain: array - Markov chain of error variance estimates.
    """

    n_par = len(theta0)
    chain = np.zeros((n_par, M))
    s2chain = np.zeros(M)
    n_y = len(data)

    if covar is None:
        covar = np.eye(n_par) * 0.1 * np.abs(theta0)

    # Functions for sum of squares and likelihood
    def ss_func(param, data):
        return np.sum((f(param) - data)**2)

    def lik_func(ss, s2):
        return np.exp(-ss / (2 * s2)) * (2 * np.pi * s2)**(-n_y / 2)

    chain[:, 0] = theta0
    ss_old = ss_func(theta0, data)
    sig_s = ss_old / (n_y - n_par)  # Estimate of error variance
    lik_old = lik_func(ss_old, sig_s)
    prior_old = prior_F(theta0)
    R = np.linalg.cholesky(covar)

    # Error variance hyperparameters for inverse gamma
    n_s = 0.01  # For inverse gamma
    aval = 0.5 * (n_s + n_y)  # This is constant
    bval = 0.5 * (n_s * sig_s + ss_old)
    s2chain[0] = invgamma.rvs(a=aval, scale=bval)

    num_acc = 0
    prob_acc = np.random.uniform(0, 1, M)

    for i in range(1, M):
        theta_star = chain[:, i - 1] + np.dot(R, np.random.normal(0, 1, n_par))

        # Check bounds
        if np.any(theta_star > UB) or np.any(theta_star < LB):
            ss_star = -np.inf
            lik_star = -np.inf
        else:
            ss_star = ss_func(theta_star, data)
            lik_star = lik_func(ss_star, s2chain[i - 1])
            lik_old = lik_func(ss_old, s2chain[i - 1])  # Recompute likelihood for comparison

        prior_star = prior_F(theta_star)
        acc_prob = (prior_star * lik_star) / (prior_old * lik_old)

        if acc_prob > prob_acc[i]:
            chain[:, i] = theta_star
            ss_old = ss_star
            lik_old = lik_star
            prior_old = prior_star
            num_acc += 1
        else:
            chain[:, i] = chain[:, i - 1]

        # Update sigma squared using inverse gamma
        bval = 0.5 * (n_s * sig_s + ss_old)
        s2chain[i] = invgamma.rvs(a=aval, scale=bval)

    print('Acceptance rate:', (num_acc / M) * 100)

    return chain, s2chain

In [ ]:
# Define the model
f_mod = lambda q: call_SIR_model(q, IC, tspace)
true_signal = f_mod(param_star)

# Generate noisy data
data = true_signal + np.random.normal(0, np.sqrt(noise_var), size=n_xpts)

# Define constant covariance
covar = np.eye(num_param) * s_theta

# MCMC algorithm parameters
M = 10000
theta0 = np.array(param_star) * 1.0

In [ ]:
# Define prior distribution
def prior_unif(param, a, b):
    """Uniform prior distribution."""
    return 1. / np.prod(b - a)

prior_F = lambda param: prior_unif(param, LB_uni, UB_uni)


In [ ]:
# Get results
chain, s2 = Metropolis_errupdate(f_mod, data, prior_F, theta0, M, covar, UB_uni, LB_uni)

In [ ]:

# Plot results
plt.figure(figsize=(12, 6))
for i in range(4):
    plt.subplot(2, 3, i + 1)
    plt.plot(chain[i, :])
    plt.axhline(param_star[i], color='k', linestyle='--', linewidth=2)
    plt.title(f'Uniform Chain Parameter {i+1}')
plt.tight_layout()
plt.show()

plt.figure(figsize=(12, 4))
for i in range(4):
    plt.subplot(2, 3, i + 1)
    plt.hist(chain[i, :], bins=50, density=True, alpha=0.5, label="Uniform")
    plt.axvline(param_star[i], color='k', linestyle='--', linewidth=2)
    plt.title(f'Parameter {i+1} Density')
    plt.legend()


In [ ]:
# Initialize variables
MAP= np.zeros(num_param)

# Pairwise plot
plt.figure()
plt.clf()
for i in range(4):
    for j in range(i+1, 4):
        plt.subplot(3, 3, 3*(i)+j)
        plt.plot(chain[j, :], chain[i, :], 'o', label='Const Covar')
        plt.legend()

    # Get density values for constant covariance chain
    kde_const = gaussian_kde(chain[i, :])
    x_const = np.linspace(min(chain[i, :]), max(chain[i, :]), 1000)
    f_const = kde_const(x_const)

    # Find the maximum of the PDF
    MAP_const[i] = x_const[np.argmax(f_const)]


# Display results
print("MAP - Const:", MAP_const)
print("True Params:", param_star)




In [ ]:
# Plot data against MAP estimates
plt.figure()
plt.clf()
plt.plot(tspace, f_mod(MAP_const), label="MAP - Const", linewidth=2)
plt.plot(tspace, data, 'ko', label="Data")
plt.plot(tspace, true_signal, label="True Signal", linewidth=2)
plt.legend()
